# TSQVT Gravitational Wave Echoes: Interactive Analysis

This notebook demonstrates the complete analysis pipeline for searching gravitational wave echoes predicted by Twistorial Spectral Quantum Vacuum Theory (TSQVT) in LIGO/Virgo data.

**Author:** Mohamed H.M. Makraini  
**Date:** November 2025

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import sys
import os

# Add src to path
sys.path.insert(0, '../src')

from data_retrieval import LIGODataRetriever
from echo_model import TSQVTEchoModel
from matched_filter import MatchedFilterSearcher

# Plotting setup
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Download and Preprocess LIGO Data

We start by downloading GW150914 data from the LIGO Open Science Center.

In [ ]:
# Initialize data retriever
retriever = LIGODataRetriever(cache_dir='../data/raw/')

# Download GW150914
event_name = 'GW150914'
data_dict = retriever.download_event_data(event_name, segment_duration=32)

print(f"Downloaded detectors: {list(data_dict.keys())}")

In [ ]:
# Preprocess H1 data
strain_h1 = data_dict['H1']
processed_h1 = retriever.preprocess_pipeline(
    strain_h1,
    flow=30.0,
    fhigh=500.0,
    whiten=True,
    remove_glitches=True
)

print(f"Processed data: {len(processed_h1)} samples at {processed_h1.sample_rate.value} Hz")

In [ ]:
# Plot raw vs processed data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Raw data
ax1.plot(strain_h1.times, strain_h1.value, 'k-', linewidth=0.5)
ax1.set_ylabel('Raw Strain', fontsize=12)
ax1.set_title(f'{event_name} H1: Raw Data', fontsize=14)
ax1.grid(True, alpha=0.3)

# Processed data
ax2.plot(processed_h1.times, processed_h1.value, 'b-', linewidth=0.5)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Whitened Strain', fontsize=12)
ax2.set_title('Preprocessed Data (whitened, bandpassed)', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Generate TSQVT Echo Template

We now generate the echo template predicted by TSQVT.

In [ ]:
# Event parameters
event_info = retriever.get_event_info(event_name)
M_total = event_info['mass_1'] + event_info['mass_2']
q = event_info['mass_1'] / event_info['mass_2']

print(f"Event parameters:")
print(f"  Total mass: {M_total:.1f} M_sun")
print(f"  Mass ratio: {q:.2f}")

# TSQVT parameters
M_star = 1.0  # GeV
xi = 0.1
gamma = 0.05
beta = 0.02

# Initialize model
echo_model = TSQVTEchoModel(
    M_total=M_total,
    M_star=M_star,
    xi=xi,
    gamma=gamma,
    beta=beta,
    q=q
)

print(f"\nRingdown properties:")
print(f"  QNM frequency: {echo_model.f_qnm:.1f} Hz")
print(f"  QNM damping time: {echo_model.tau_qnm*1000:.2f} ms")

In [ ]:
# Compute echo predictions
n_echoes = 3

print("\nTSQVT Echo Predictions:")
print("-" * 60)
print(f"{'Echo':<6} {'Delay (ms)':<12} {'Amplitude':<12} {'Freq Shift':<12}")
print("-" * 60)

for n in range(1, n_echoes + 1):
    dt = echo_model.echo_delay(n)
    A = echo_model.echo_amplitude(n)
    df = echo_model.frequency_shift(n)
    print(f"{n:<6} {dt*1000:<12.1f} {A:<12.3f} {df:<12.4f}")

In [ ]:
# Generate template
sample_rate = int(processed_h1.sample_rate.value)
duration = float(processed_h1.duration.value)

template_times, template_strain = echo_model.generate_echo(
    t_merger=duration / 2,
    n_echoes=n_echoes,
    sample_rate=sample_rate,
    duration=duration,
    include_ringdown=False  # Only echoes
)

print(f"\nTemplate generated: {len(template_strain)} samples")

In [ ]:
# Plot template
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(template_times, template_strain, 'b-', linewidth=1)
ax.axvline(duration / 2, color='r', linestyle='--', alpha=0.5, label='Merger')

# Mark echo times
for n in range(1, n_echoes + 1):
    t_echo = duration / 2 + echo_model.echo_delay(n)
    ax.axvline(t_echo, color='g', linestyle=':', alpha=0.7, label=f'Echo {n}' if n == 1 else '')

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Strain', fontsize=12)
ax.set_title(f'TSQVT Echo Template (M_*={M_star} GeV, ξ={xi}, γ={gamma})', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Matched Filter Analysis

We perform a matched filter search for echoes in the processed data.

In [ ]:
# Initialize matched filter searcher
searcher = MatchedFilterSearcher(
    sample_rate=sample_rate,
    fmin=30.0,
    fmax=500.0
)

# Convert data to numpy array
data_array = processed_h1.value

# Perform search
search_window = (0.1, 2.0)  # seconds after merger
snr_threshold = 4.0

results = searcher.search_for_echoes(
    data_array,
    template_strain,
    search_window=search_window,
    snr_threshold=snr_threshold,
    t_merger=duration / 2
)

print(f"\nSearch Results:")
print(f"  Max SNR: {results['max_snr']:.2f}")
print(f"  Candidates above threshold: {results['n_candidates']}")

In [ ]:
# Display candidate echoes
if results['n_candidates'] > 0:
    print("\nCandidate Echoes:")
    print("-" * 60)
    print(f"{'#':<4} {'Time (s)':<12} {'Δt (ms)':<12} {'SNR':<8}")
    print("-" * 60)
    
    for i, (t, snr) in enumerate(zip(results['peaks'][:10], results['peak_snrs'][:10])):
        dt_merger = (t - duration / 2) * 1000
        print(f"{i+1:<4} {t:<12.3f} {dt_merger:<12.1f} {snr:<8.2f}")
else:
    print("\nNo candidates found above threshold.")

In [ ]:
# Plot SNR timeseries
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(results['times'], results['snr'], 'k-', linewidth=0.5, alpha=0.7, label='SNR')
ax.axhline(snr_threshold, color='r', linestyle='--', linewidth=2, label=f'Threshold ({snr_threshold})')
ax.axvline(duration / 2, color='g', linestyle='--', alpha=0.5, label='Merger')

# Mark predicted echo times
for n in range(1, n_echoes + 1):
    t_echo = duration / 2 + echo_model.echo_delay(n)
    ax.axvline(t_echo, color='orange', linestyle=':', alpha=0.7)

# Mark detected candidates
if results['n_candidates'] > 0:
    ax.plot(results['peaks'], results['peak_snrs'], 'ro', markersize=8, label='Detected', zorder=5)

# Highlight search window
window_start = duration / 2 + search_window[0]
window_end = duration / 2 + search_window[1]
ax.axvspan(window_start, window_end, alpha=0.1, color='blue', label='Search window')

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('SNR', fontsize=12)
ax.set_title(f'{event_name} H1: Matched Filter SNR', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Statistical Significance

Compute the statistical significance of detected candidates.

In [ ]:
# Compute significance
p_value, sigma = searcher.compute_significance(results['snr'])

print(f"\nStatistical Significance:")
print(f"  p-value: {p_value:.4e}")
print(f"  Significance: {sigma:.2f} σ")

if sigma > 5.0:
    print(f"\n  *** DISCOVERY (>5σ) ***")
elif sigma > 3.0:
    print(f"\n  *** EVIDENCE (>3σ) ***")
elif sigma > 2.0:
    print(f"\n  Marginal evidence ({sigma:.1f}σ)")
else:
    print(f"\n  No significant detection")

## 5. Parameter Estimation (Optional)

If echoes are detected, estimate TSQVT parameters.

In [ ]:
# This cell is optional and computationally intensive
# Uncomment to run parameter estimation

# param_bounds = {
#     'M_star': (0.1, 10.0),  # GeV
#     'xi': (0.01, 0.5),
#     'gamma': (0.01, 0.2)
# }

# best_fit = echo_model.parameter_estimation(
#     data_array,
#     template_times,
#     param_bounds,
#     n_iterations=100
# )

# print("\nBest-fit TSQVT parameters:")
# print(f"  M_* = {best_fit['M_star']:.2f} GeV")
# print(f"  ξ = {best_fit['xi']:.3f}")
# print(f"  γ = {best_fit['gamma']:.3f}")
# print(f"  -log L = {best_fit['neg_log_L']:.2f}")

## 6. Summary

Recap of analysis results.

In [ ]:
print("=" * 70)
print(f"ANALYSIS SUMMARY: {event_name}")
print("=" * 70)
print(f"\nEvent Parameters:")
print(f"  Total mass: {M_total:.1f} M_sun")
print(f"  Mass ratio: {q:.2f}")
print(f"\nTSQVT Parameters:")
print(f"  M_* = {M_star} GeV")
print(f"  ξ = {xi}")
print(f"  γ = {gamma}")
print(f"  β = {beta}")
print(f"\nSearch Configuration:")
print(f"  Window: {search_window[0]:.2f} - {search_window[1]:.2f} s")
print(f"  Threshold: SNR > {snr_threshold}")
print(f"\nResults:")
print(f"  Max SNR: {results['max_snr']:.2f}")
print(f"  Candidates: {results['n_candidates']}")
print(f"  Significance: {sigma:.2f} σ (p={p_value:.2e})")
print("=" * 70)